In [1]:
import pandas as pd

# Chemin vers ton fichier local
path = "dataset/BASE_TD_FILO_IRIS_2021_DEC.csv"

# L’Insee utilise en général :
# - séparateur ; 
# - virgule décimale ,
df = pd.read_csv(path, sep=";", decimal=",")

# Aperçu du dataset
print(df.head())
print(df.info())

        IRIS DEC_PIMP21 DEC_TP6021 DEC_INCERT21 DEC_Q121 DEC_MED21 DEC_Q321  \
0  010040101       43,0       29,0            2    12610     19330    26390   
1  010040102       42,0       39,0            1     9730     16830    24620   
2  010040201       47,0       29,0            1    12220     19940    27650   
3  010040202       62,0       14,0            1    18350     25560    35010   
4  010330102       42,0       31,0            1    11280     19870    30050   

  DEC_EQ21 DEC_D121 DEC_D221  ... DEC_RD21 DEC_S80S2021 DEC_GI21 DEC_PACT21  \
0     0,71     7760    11300  ...      4,4          6,3    0,318       69,3   
1     0,89     4680     8600  ...      7,1          9,0    0,362       70,5   
2     0,77     6500    10300  ...      5,7          8,0    0,352       69,3   
3     0,65    11960    16450  ...      3,8          6,3    0,360       65,5   
4     0,94     5250     9790  ...      8,5         10,6    0,390       74,1   

  DEC_PTSA21 DEC_PCHO21 DEC_PBEN21 DEC_PPEN21 DEC_

In [2]:
# Remplacer les codes manquants ('..', 'NA', 'ND') si présents
df = df.replace(["..", "NA", "ND"], pd.NA)

# Conversion automatique des colonnes qui peuvent devenir numériques
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors="ignore")

print(df.head())

        IRIS DEC_PIMP21 DEC_TP6021 DEC_INCERT21 DEC_Q121 DEC_MED21 DEC_Q321  \
0  010040101       43,0       29,0            2    12610     19330    26390   
1  010040102       42,0       39,0            1     9730     16830    24620   
2  010040201       47,0       29,0            1    12220     19940    27650   
3  010040202       62,0       14,0            1    18350     25560    35010   
4  010330102       42,0       31,0            1    11280     19870    30050   

  DEC_EQ21 DEC_D121 DEC_D221  ... DEC_RD21 DEC_S80S2021 DEC_GI21 DEC_PACT21  \
0     0,71     7760    11300  ...      4,4          6,3    0,318       69,3   
1     0,89     4680     8600  ...      7,1          9,0    0,362       70,5   
2     0,77     6500    10300  ...      5,7          8,0    0,352       69,3   
3     0,65    11960    16450  ...      3,8          6,3    0,360       65,5   
4     0,94     5250     9790  ...      8,5         10,6    0,390       74,1   

  DEC_PTSA21 DEC_PCHO21 DEC_PBEN21 DEC_PPEN21 DEC_

C:\Users\sppre\AppData\Local\Temp\ipykernel_3092\218324109.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors="ignore")


In [3]:
df.describe()

,IRIS,DEC_PIMP21,DEC_TP6021,DEC_INCERT21,DEC_Q121,DEC_MED21,DEC_Q321,DEC_EQ21,DEC_D121,DEC_D221,...,DEC_RD21,DEC_S80S2021,DEC_GI21,DEC_PACT21,DEC_PTSA21,DEC_PCHO21,DEC_PBEN21,DEC_PPEN21,DEC_PAUT21,DEC_NOTE21
count,16026,16026,16026,16026,16026,16026,16026,16026,16026,16026,...,16026,16026,16026,16026,16026,16026,16026,16026,16026,16026
unique,16026,85,88,3,2737,3498,4343,192,1953,2573,...,818,861,468,607,604,114,169,520,378,4
top,974240105,ns,ns,1,ns,ns,ns,ns,ns,ns,...,so,so,ns,ns,ns,ns,ns,ns,ns,0
freq,1,1209,1209,8835,1209,1209,1209,1209,1209,1209,...,2219,1577,1209,1209,1211,1211,1211,1209,1209,14325
